In [10]:
import json
import time
import datetime
import requests
import math
import random

from solc import compile_source      #$pip install py-solc
from web3.contract import ConciseContract

from web3 import Web3, KeepAliveRPCProvider, IPCProvider, HTTPProvider

web3 = Web3(HTTPProvider("http://127.0.0.1:8545", request_kwargs={'timeout': 600}))

coinbase = web3.eth.coinbase
web3.eth.defaultBlock = "latest"
transaction = {'from': coinbase}

Fresh Deployment of the Contracts

In [12]:
#OLI Origin
Oliorigin_abi_str  = '[{"constant":true,"inputs":[{"name":"_account","type":"address"}],"name":"get_oliType","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_account","type":"address"}],"name":"get_oliCkt","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"kill","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_account","type":"address"}],"name":"get_oliTrafoid","outputs":[{"name":"","type":"uint32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tid","type":"uint32"}],"name":"get_gsoAddr","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_account","type":"address"},{"name":"_index","type":"uint8"}],"name":"get_oliPeakLoad","outputs":[{"name":"","type":"uint16"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"oli","type":"address"},{"name":"lat","type":"uint32"},{"name":"long","type":"uint32"},{"name":"trafo","type":"uint32"},{"name":"ckt","type":"uint8"},{"name":"typex","type":"uint8"},{"name":"pload","type":"uint16[]"}],"name":"addOli","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"paymentAddress","type":"address"},{"indexed":false,"name":"latOfLocation","type":"uint32"},{"indexed":false,"name":"longOfLocation","type":"uint32"}],"name":"newAddedOli","type":"event"}]'
Oliorigin_abi      = json.loads(Oliorigin_abi_str)
Oliorigin_source_code = '''
pragma solidity ^0.4.11;

import "github.com/ethereum/solidity/std/mortal.sol";

contract OliOrigin is mortal{
  
    mapping (address => Details) OliAddressMapping;
    mapping (uint32 => address) GSO;


    event newAddedOli(address paymentAddress, uint32 latOfLocation, uint32 longOfLocation);
    
    struct Details {
        uint32 latitude;
        uint32 longitude;
        uint32 trafoid;
        uint8 ckt; //In case of Prosumers' ckt rep. --> 1st ckt:0; 2nd ckt:1; In case of Trafo 4branches --> 4 ckt
        uint8 _type; //0:PV, 1:Wind, 2:CCP, 3:CHP, 4:Coal, 5:battery+, 6: battery-, 7:Consumer, 8:DNO
        uint16[] _pload; // if _type=8 then index:0->trafoload
    }

    function addOli(address oli, uint32 lat, uint32 long, uint32 trafo, uint8 ckt, uint8 typex, uint16[] pload) onlyowner{
        OliAddressMapping[oli] = Details(lat, long, trafo, ckt, typex, pload);
        newAddedOli(oli, lat, long);
        if (typex==8){
            GSO[trafo]=oli;
        }
    }

    //Oli types passing to Daughter Auction
    function get_oliType(address _account) constant returns (uint8) {
        return OliAddressMapping[_account]._type;
    }
    //Oli amount passing to Daughter Auction
    function get_oliPeakLoad(address _account, uint8 _index) constant returns (uint16) {
        return OliAddressMapping[_account]._pload[_index];
    }

    function get_oliCkt(address _account) constant returns (uint8) {
        return OliAddressMapping[_account].ckt;
    }

    function get_oliTrafoid(address _account) constant returns (uint32) {
        return OliAddressMapping[_account].trafoid;
    }
    function get_gsoAddr(uint32 _tid) constant returns (address) {
        return GSO[_tid];
    }
    
}
'''
olicoin_compiled_sol = compile_source(Oliorigin_source_code)

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

In [15]:
compile_source

<function solc.main.compile_source>